In [5]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, StepSequence
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails

ws = Workspace.from_config()
blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'drift_pipeline'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = "vdsm-cluster-d13"  # ALL SMALL LETTER, NO underscore, 16ws long only.
aml_compute = AmlCompute(ws, aml_compute_target)

# Use an existing env which was registered to the ws
env = Environment.get(workspace=ws,name="vdsm_all",version="5")

In [7]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

script_run_config = RunConfiguration()
script_run_config.target = aml_compute
script_run_config.environment = env

veg_type_pipeline_param = PipelineParameter(name="veg_type", default_value='bs')

example_step = PythonScriptStep(
    name="drift_pipeline",
    script_name="drift_pipeline_dev.py",
    arguments=[
        "--veg_type", veg_type_pipeline_param
    ],
    runconfig=script_run_config,
    source_directory="."
    )

step_sequence = StepSequence(steps=[example_step])

pipeline = Pipeline(workspace=ws, steps=step_sequence)
pipeline_run = exp.submit(pipeline)
print('Datadrift pipeline submitted for execution')
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

Created step drift_pipeline [393c7ba9][e467014c-f0db-4d5f-9d21-b8d465cdce26], (This step will run and generate new outputs)
Submitted PipelineRun 5044fa12-be42-472d-8eda-13d27ddb0583
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5044fa12-be42-472d-8eda-13d27ddb0583?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
Datadrift pipeline submitted for execution


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 5044fa12-be42-472d-8eda-13d27ddb0583
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5044fa12-be42-472d-8eda-13d27ddb0583?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: b2312c53-0c2b-4330-9186-5dd933cf16d1
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b2312c53-0c2b-4330-9186-5dd933cf16d1?wsid=/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourcegroups/arg-rt-pacedev-spacetech/workspaces/aml-ws-spacetech001&tid=4341df80-fbe6-41bf-89b0-e6e2379c9c23
StepRun( drift_pipeline ) Status: Running

StepRun(drift_pipeline) Execution Summary
StepRun( drift_pipeline ) Status: Failed

Warnings:
{
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "AzureMLCompute job failed.\nExecutionFailed: [REDACTED]\n\texit_codes: 1",
    "me

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{'code': ExecutionFailed, 'message': [{\"exit_code\":1,\"error_message\":\"Execution failed with error: [stderr]    return ClientBase._execute_func_internal(\\n[stderr]  File \\\"/azureml-envs/azureml_305f81c4727c08535a503538ab343409/lib/python3.8/site-packages/azureml/_restclient/clientbase.py\\\", line 367, in _execute_func_internal\\n[stderr]    left_retry = cls._handle_retry(back_off, left_retry, total_retry, error, logger, func)\\n[stderr]  File \\\"/azureml-envs/azureml_305f81c4727c08535a503538ab343409/lib/python3.8/site-packages/azureml/_restclient/clientbase.py\\\", line 426, in _handle_retry\\n[stderr]    raise error\\n[stderr]  File \\\"/azureml-envs/azureml_305f81c4727c08535a503538ab343409/lib/python3.8/site-packages/azureml/_restclient/clientbase.py\\\", line 358, in _execute_func_internal\\n[stderr]    response = func(*args, **kwarg

KeyError: 'log_files'